# Customer Churn EDA & Prediction

1. Questions asked
2. Datasets used
3. ETL process
4. Technologies used
5. Data Preprocessing
6. Logistic Regression (ML)
7. Conclusions

## 0.5 Importing Libraries & Dependencies

In [4]:
import json
import pandas as pd
import numpy as np
import joblib
import os
import pymysql
from datetime import datetime
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 1. Questions Asked

##  5. Data Preprocessing

In [5]:
## Get credentials from environment variables
host = os.environ['DB_HOST']
port = os.environ['DB_PORT']
user = os.environ['DB_USER']
password = os.environ['DB_PASSWORD']
database = os.environ['DB_NAME']

In [6]:
# Extract data from mysql db

## Create sqlalchemy engine for db connection
try:
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)
    print('Successfully connected to MySQL database')
    
except Exception as e:
    print(f"Error connecting to database: {e}")
    

Successfully connected to MySQL database


In [7]:
## Read in sql query (it's kind of a monster, so I exciled it to its own file)
with open('aggregation_query.sql', 'r') as f:
    query = f.read()
    

In [11]:
## Extract data
try:
    df = pd.read_sql(query, engine)
    print(f"Loaded {len(df)} rows and {len(df.columns)} columns")
    print(df.describe())
    
except Exception as e:
    print(f"Error loading data: {e}")
    

Loaded 7043 rows and 36 columns
       tenure_months          age  is_under_30  is_senior_citizen  \
count    7043.000000  6832.000000  6832.000000         6832.00000   
mean       32.371149    46.527664     0.198917            0.16291   
std        24.559481    16.763525     0.399215            0.36931   
min         0.000000    19.000000     0.000000            0.00000   
25%         9.000000    32.000000     0.000000            0.00000   
50%        29.000000    46.000000     0.000000            0.00000   
75%        55.000000    60.000000     0.000000            0.00000   
max        72.000000    80.000000     1.000000            1.00000   

       has_partner  has_dependents  number_of_dependents     population  \
count  6832.000000     6832.000000           6832.000000    7043.000000   
mean      0.481265        0.228923              0.464286   22139.814568   
std       0.499685        0.420170              0.958767   21152.174407   
min       0.000000        0.000000            

There appear to be quite a few missing values post-ETL, so we'll need to handle that in our preprocessing (via Imputing) along with the categorical values, numerical standardization, and splitting the data into training and test sets.

In [18]:
## Taking a look at a summary of missing values
missing = df.isna().sum()
not_missing = df.notna().sum()

pd.DataFrame({
    'missing': missing,
    'not missing': not_missing,
    'total': len(df)
})

,missing,not missing,total
customer_id,0,7043,7043
tenure_months,0,7043,7043
age,211,6832,7043
is_under_30,211,6832,7043
is_senior_citizen,211,6832,7043
has_partner,211,6832,7043
has_dependents,211,6832,7043
number_of_dependents,211,6832,7043
city,0,7043,7043
zip_code,0,7043,7043


In [17]:
## Impute missing values




,missing,not missing,total
customer_id,0,7043,7043
tenure_months,0,7043,7043
age,211,6832,7043
is_under_30,211,6832,7043
is_senior_citizen,211,6832,7043
has_partner,211,6832,7043
has_dependents,211,6832,7043
number_of_dependents,211,6832,7043
city,0,7043,7043
zip_code,0,7043,7043


## 6. Logistic Regression Model